In [5]:
import argparse
from ase.io import read
from ase.cluster import Octahedron
from ase.optimize import BFGS
from mace.calculators import MACECalculator
from acat.ga.adsorbate_operators import (AddAdsorbate, RemoveAdsorbate,
                                        MoveAdsorbate, ReplaceAdsorbate,
                                        SimpleCutSpliceCrossoverWithAdsorbates)
from acat.ga.particle_mutations import (RandomPermutation, COM2surfPermutation,
                                        Rich2poorPermutation, Poor2richPermutation)
from ase.ga.offspring_creator import OperationSelector
from acat.adsorption_sites import ClusterAdsorptionSites
from acat.adsorbate_coverage import ClusterAdsorbateCoverage

In [6]:
from npl.monte_carlo.ensembles.canonical_ensemble import CanonicalEnsemble

In [7]:
atoms = Octahedron('Ag', 6,1)
atoms.set_cell([25,25,25])
atoms.center()

sas = ClusterAdsorptionSites(atoms, composition_effect=True)

In [9]:
atoms = Octahedron('Ag', 6,1)
atoms.set_cell([25,25,25])
atoms.center()

sas = ClusterAdsorptionSites(atoms, composition_effect=True)
species = ['O']

add = AddAdsorbate(species, adsorption_sites=sas, num_muts=1)
for i in range(5):
    atoms.info['data'] = {'tag': None}
    atoms.info['confid'] = 1
    atoms, _ = add.get_new_individual([atoms])

calculator = MACECalculator(
    model_paths='/home/riccardo/Downloads/mace_agnesi_medium.model',
    device = 'cpu'
    )
optimizer = BFGS
fmax = 0.5
steps = 200000
traj_file = f'{atoms.get_chemical_formula()}.traj'
sas = ClusterAdsorptionSites(atoms, composition_effect=True)
soclist = ([1],
    [
    MoveAdsorbate(['O'], adsorption_sites=sas, num_muts=1)
    ])
op_list = OperationSelector(*soclist)

outfile = f'{atoms.get_chemical_formula()}.out'
outfile_write_interval = 10
random_seed = 123

montecarlo = CanonicalEnsemble(atoms,
                calculator,
                random_seed,
                optimizer, 
                fmax,
                300, 
                steps, 
                traj_file, 
                op_list, 
                outfile,
                outfile_write_interval)

montecarlo.run()

/home/riccardo/anaconda3/envs/np-lib/lib/python3.9/site-packages/mace/calculators/mace.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


No dtype selected, switching to float64 to match model dtype.
